In [ ]:
import os
import pandas as pd
import numpy as np

import torch

from google.colab import drive

# Mount Drive
drive.mount("/content/drive")

# Project root on Drive
colab_dir = "/content/drive/MyDrive/NLP/respect-cfpb"

# Load pre processed splits
train_df = pd.read_csv(os.path.join(colab_dir, "data/processed/train.csv"))
val_df   = pd.read_csv(os.path.join(colab_dir, "data/processed/val.csv"))
test_df  = pd.read_csv(os.path.join(colab_dir, "data/processed/test.csv"))

print("Train shape:", train_df.shape)
print("Val shape:  ", val_df.shape)
print("Test shape: ", test_df.shape)

print("\nColumns:", list(train_df.columns))

print("\nTrain risk_flag value counts:")
print(train_df["risk_flag"].value_counts(normalize=True).rename("proportion"))